In [3]:
#HW 1 Paint

import cv2
import numpy as np

bgr = [0, 255, 0]
radius = 5
rectSize = 100
light = 255

canvas = np.zeros((600, 600, 3), dtype = np.uint8)
colorSpectrum = np.zeros((rectSize * 3, rectSize, 3), dtype = np.uint8)

def showInstruments(surface):
    for i in range(3):
        cv2.rectangle(surface, (0, i * rectSize), (rectSize, (i + 1) * rectSize), [255 * int(i == j) for j in range(3)], -1)
    cv2.imshow("instruments", surface)
    
    for i in range(3):
        print([i * rectSize, (i + 1) * rectSize], [0, rectSize])
    
instrument = "pen"

pressed = False
ix, iy = 0, 0

instToMeth = {
    "rect" : cv2.rectangle,
    "line" : cv2.line
}

def paint(event, x, y, flags, params):
    
    global ix, iy, canvas, pressed, rgb, light
    
    for i in range(3):
        if bgr[i] > 0:
            bgr[i] = light
        
    if instrument == "pen":
        if flags == cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(canvas, (x, y), radius, bgr, -1)

        cv2.imshow("canvas", canvas)
        
    elif instrument == "rect":
        precanvas = canvas.copy()
        
        width = radius if instrument == "line" else -1
        
        if pressed and flags == cv2.EVENT_FLAG_LBUTTON:
            instToMeth[instrument](precanvas, (min(x, ix), min(y, iy)), (max(x, ix), max(y, iy)), bgr, width)
        elif pressed:
            pressed = False
            instToMeth[instrument](precanvas, (min(x, ix), min(y, iy)), (max(x, ix), max(y, iy)), bgr, width)
            canvas = precanvas
        elif not pressed and flags == cv2.EVENT_FLAG_LBUTTON:
            pressed = True
            ix, iy = x, y
        cv2.imshow("canvas", precanvas)
    
    elif instrument == "line":
        precanvas = canvas.copy()
        
        width = radius if instrument == "line" else -1
        
        if pressed and flags == cv2.EVENT_FLAG_LBUTTON:
            instToMeth[instrument](precanvas, (ix, iy), (x, y), bgr, width)
        elif pressed:
            pressed = False
            instToMeth[instrument](precanvas, (ix, iy), (x, y), bgr, width)
            canvas = precanvas
        elif not pressed and flags == cv2.EVENT_FLAG_LBUTTON:
            pressed = True
            ix, iy = x, y
        cv2.imshow("canvas", precanvas)
    
    
def choiceHandle(event, x, y, flags, params):
    global bgr
    for i in range(3):
        if event == cv2.EVENT_LBUTTONDOWN and y in range(i * rectSize, (i + 1) * rectSize) and x in range(0, rectSize):
            bgr = [255 * int(i == j) for j in range(3)]

cv2.namedWindow("canvas")
cv2.namedWindow("instruments")

cv2.resizeWindow("instruments", colorSpectrum.shape[0], colorSpectrum.shape[1])

showInstruments(colorSpectrum)

cv2.setMouseCallback("canvas", paint)
cv2.setMouseCallback("instruments", choiceHandle)

while True:
    k = cv2.waitKey(0)
    
    if k == ord('q'):
        break
    elif k == ord('0'):
        instrument = 'pen'
    elif k == ord('1'):
        instrument = 'rect'
    elif k == ord('2'):
        instrument = 'line'
    elif k == ord('+'):
        radius += 1
        radius = min(40, radius)
    elif k == ord('-'):
        radius -= 1
        radius = max(0, radius)
    elif k == ord('w'):
        light += 5
        light = min(light, 255)
    elif k == ord('e'):
        light -= 5
        light = max(light, 0)

cv2.destroyAllWindows()

[0, 100] [0, 100]
[100, 200] [0, 100]
[200, 300] [0, 100]


In [1]:
#HW Camera work (Projects 1, 2, 3)

import matplotlib.pyplot as plt
import cv2
import numpy as np
vid = cv2.VideoCapture(0)

ret, frame = vid.read()

x, y = frame.shape[0] // 2, frame.shape[1] // 2

width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
print(x, y, frame[:x, :y].shape)

%matplotlib inline

cv2.namedWindow("frame")

result = cv2.VideoWriter("cv_result.avi", cv2.VideoWriter_fourcc(*'XVID'), 60, (width, height))

mpos = None

prepos = None

released = True

myFilter = lambda x: np.array(map(max(255, x * 2), x))

filters = [
    lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR),
    lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HSV),
    lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HLS),
    lambda x: cv2.blur(x, [40, 40]),
]

filtChoice = 0

rects = [[[0, 100], [100, 200], 1]]

def mouseCallback(event, x, y, flags, params):
    global prepos, mpos, frame, ret, result, released, rects
    
    if flags == cv2.EVENT_FLAG_LBUTTON and released:
        mpos = None
        prepos = None
        released = False
    elif flags == cv2.EVENT_FLAG_LBUTTON and not prepos:
        prepos = [max(min(x, frame.shape[1]), 0), max(min(y, frame.shape[0]), 0)]
        mpos = [max(min(x, frame.shape[1]), 0), max(min(y, frame.shape[0]), 0)]
    elif flags == cv2.EVENT_FLAG_LBUTTON:
        mpos = [max(min(x, frame.shape[1]), 0), max(min(y, frame.shape[0]), 0)]
    elif flags != cv2.EVENT_FLAG_LBUTTON:
        if mpos and prepos:
            mnx, mny, mxx, mxy = min(prepos[0], mpos[0]), min(prepos[1], mpos[1]), max(prepos[0], mpos[0]), max(prepos[1], mpos[1])
            rects.append([[mnx, mny], [mxx, mxy], filtChoice])
        released = True

cv2.setMouseCallback("frame", mouseCallback)

while True:
    global frame, ret, mpos, prepos

    ret, preframe = vid.read()

    frame = preframe.copy()

    frame[:x, :y] = cv2.cvtColor(preframe[:x, :y], cv2.COLOR_RGB2BGR)
    frame[x:, :y] = cv2.blur(preframe[x:, :y], [40, 40])
    frame[:x, y:] = cv2.cvtColor(preframe[:x, y:], cv2.COLOR_RGB2HSV)

    for rect in rects:
        try:
            if rect[0][0] != rect[1][0] and rect[0][1] != rect[1][1]:
                frame[rect[0][1]:rect[1][1], rect[0][0]:rect[1][0]] = filters[rect[2]](preframe[rect[0][1]:rect[1][1], rect[0][0]:rect[1][0]])
        except:
            print(rect)
            print("LOL HERE")
    
    if mpos and prepos:
        mnx, mny, mxx, mxy = min(prepos[0], mpos[0]), min(prepos[1], mpos[1]), max(prepos[0], mpos[0]), max(prepos[1], mpos[1])
        if mnx != mxx and mxy != mny:
            frame[mny:mxy, mnx:mxx] = filters[filtChoice](preframe[mny:mxy, mnx:mxx])
            cv2.rectangle(frame,[mnx, mny], [mxx, mxy], (0, 255, 255), 3)
    
    k = cv2.waitKey(1)
#     frame[x:, y:] =

    result.write(frame)
    
    cv2.imshow("frame", frame)
    
    if k == ord('q'):
        break
        
    if k == ord('c'):
        mpos = None
        prepos = None
        rects = []
        
    if k in [ord('0') + i for i in range(4)]:
        filtChoice = k - ord('0')
        
result.release() 
vid.release()
cv2.destroyAllWindows()

240 320 (240, 320, 3)


In [10]:
import cv2

vid = cv2.VideoCapture("cv_result.avi")

while True:
    try:
        frame, ret = vid.read()
        
        cv2.imshow("lol", frame)
        
        k = cv2.waitKey(1)
        
        if k == ord('q'):
            break
    except:
        break
        
vid.release()
cv2.destroyAllWindows()

In [ ]:
vid = cv2.VideoCapture(0)

f